##### Importation des librairies

In [2]:
#!pip install scikit-learn tensorflow pandas

  Using cached pandas-2.0.3-cp38-cp38-win_amd64.whl (10.8 MB)
  Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
  Using cached pandas-2.0.3-cp38-cp38-win_amd64.whl (10.8 MB)
  Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)


You should consider upgrading via the 'c:\users\jean marc\documents\master_these\partie_codes_saison_ii\version_2024_06_16\.train\scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor,GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from datetime import datetime, timedelta
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectFromModel
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.compose import ColumnTransformer
from config import Config, Constant
import os, sys
import shutil
import warnings
warnings.filterwarnings('ignore')

In [ ]:
resultat_compile=dict()

In [ ]:
df=pd.read_csv(os.path.join(Config.DATASET_DIR,Config.DATA_1DEK))

In [ ]:
df.head()

In [ ]:
# Créer un répertoire de sortie s'il n'existe pas déjà
dir_resultat = Config.RESULT_DIR
# Supprimer le dossier existant, y compris son contenu
if os.path.exists(dir_resultat):
    shutil.rmtree(dir_resultat)
# Créer un nouveau répertoire de sortie
os.makedirs(dir_resultat, exist_ok=True)

In [ ]:
# Créer un répertoire de sortie s'il n'existe pas déjà
dir_resultat_sel_var = Config.RESULT_DIR_SEL_VAR
# Supprimer le dossier existant, y compris son contenu
if os.path.exists(dir_resultat_sel_var):
    shutil.rmtree(dir_resultat_sel_var)
# Créer un nouveau répertoire de sortie
os.makedirs(dir_resultat_sel_var, exist_ok=True)

In [ ]:
df.head()

#### Creation des fonctions utiles

In [ ]:
# Définition de la fonction to_numeric_with_nan
def to_numeric_with_nan(value):
    try:
        return int(value)
    except (ValueError, TypeError):
        return pd.NA  # Retourne une valeur manquante

In [ ]:
def label_change(value):
    if pd.isna(value):
        return pd.NA
    elif value == 0.0:
        return '0'
    elif value == 1.0:
        return '1'
    else:
        return value  # Si la valeur est différente de NaN, 0.0 et 1.0, la renvoyer telle quelle

In [ ]:
def load_data_dek(file_path):
    df = pd.read_csv(file_path)

    # Remove rows containing NaN values
    df.dropna(axis=0, inplace=True)

    # Convert the 'Year' and 'Month' columns to numeric (if necessary)
    df[['Year', 'Month','Decade']] = df[['Year', 'Month','Decade']].applymap(to_numeric_with_nan)

    # Apply the 'label_change' function to the 'Label Secheresse' column
    df['Label Secheresse'] = df['Label Secheresse'].apply(label_change)
    # Remove the colonne Date
    #df.drop('Date', axis=1, inplace=True)
    # Encode categorical columns
    ordinal_columns = ['Saison_Pluie','Month','Label Secheresse']
    encoder = LabelEncoder()
    for col in ordinal_columns:
        df[col] = encoder.fit_transform(df[col])
    # Shift the 'Label Secheresse' column two months forward for prediction
    #df['Secheresse_future'] = df['Label Secheresse']#.shift(2)
    # Remove rows containing NaN values again
    df.dropna(axis=0, inplace=True)

    # Normalize numeric columns that are not in ordinal_columns
    #numeric_columns = [col for col in df.columns if col not in ordinal_columns]
    numeric_columns = ['v_wind_925','u_wind_850','u_wind_700','u_wind_200','eau_precipitable','t_point_rosee','h_vol_sol_wat','anom_lef_dek','anom_nino_dek']
    scaler = StandardScaler()
    df[numeric_columns] = scaler.fit_transform(df[numeric_columns])
    return df


In [ ]:
def load_data_mon(file_path):
    df = pd.read_csv(file_path)

    # Remove rows containing NaN values
    df.dropna(axis=0, inplace=True)

    # Convert the 'Year' and 'Month' columns to numeric (if necessary)
    if 'Decade' in df.columns:
        df.drop('Decade', axis=1, inplace=True)
    df[['Year', 'Month']] = df[['Year', 'Month']].applymap(to_numeric_with_nan)

    # Apply the 'label_change' function to the 'Label Secheresse' column
    df['Label Secheresse'] = df['Label Secheresse'].apply(label_change)
    # Remove the colonne Date
    #df.drop('Date', axis=1, inplace=True)
    # Encode categorical columns
    ordinal_columns = ['Saison_Pluie','Month','Label Secheresse']
    encoder = LabelEncoder()
    for col in ordinal_columns:
        df[col] = encoder.fit_transform(df[col])
    # Shift the 'Label Secheresse' column two months forward for prediction
    #df['Secheresse_future'] = df['Label Secheresse']#.shift(2)
    # Remove rows containing NaN values again
    df.dropna(axis=0, inplace=True)

    # Normalize numeric columns that are not in ordinal_columns
    #numeric_columns = [col for col in df.columns if col not in ordinal_columns]
    numeric_columns = ['v_wind_925','u_wind_850','u_wind_700','u_wind_200','eau_precipitable','t_point_rosee','h_vol_sol_wat','anom_lef_mois','anom_nino_mois']
    scaler = StandardScaler()
    df[numeric_columns] = scaler.fit_transform(df[numeric_columns])
    #df.drop(['Year','Decade'], axis=1, inplace=True)

    return df


In [ ]:
features=['Station','v_wind_925','u_wind_850','u_wind_700','u_wind_200','eau_precipitable','t_point_rosee','h_vol_sol_wat','anom_lef_dek','anom_nino_dek','Saison_Pluie']
feature_rnn_dek=['v_wind_925','u_wind_850','u_wind_700','u_wind_200','eau_precipitable','t_point_rosee','h_vol_sol_wat','anom_lef_dek','anom_nino_dek','Saison_Pluie','Label Secheresse']
feature_rnn_mon=['v_wind_925','u_wind_850','u_wind_700','u_wind_200','eau_precipitable','t_point_rosee','h_vol_sol_wat','anom_lef_mois','anom_nino_mois','Saison_Pluie','Label Secheresse']

In [ ]:
def train_and_evaluate_model_ranf(df_data,spei,type_sec,localite):
    df=df_data.copy()
    if 'Decade' in df.columns:
        df.drop('Decade', axis=1, inplace=True)
    df = df.drop(['Station', 'Year', 'Date'], axis=1)
    df['Label Secheresse'] = df['Label Secheresse'].astype(str)

    X = df.drop('Label Secheresse', axis=1)
    y = df['Label Secheresse']

    # Create a pipeline with data preprocessing, feature selection, and classifier
    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        #('scaler', StandardScaler()),
        ('feature_selector', SelectFromModel(RandomForestClassifier(random_state=42),threshold='median')),
        ('classifier', RandomForestClassifier(random_state=42))
    ])

    param_grid = {
        'classifier__n_estimators': [50, 100, 200],
        'classifier__max_depth': [None, 10, 20, 30],
        'classifier__min_samples_split': [2, 5, 10],
        'classifier__min_samples_leaf': [1, 2, 4]
    }

    grid_search = GridSearchCV(pipeline, param_grid, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
                               n_jobs=-1, verbose=2)

    grid_search.fit(X, y)

    best_model = grid_search.best_estimator_

    k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(best_model, X, y, cv=k_fold, scoring='accuracy')
    mean_accuracy = scores.mean()

    # Calculate additional metrics
    y_pred = best_model.predict(X)
    confusion = confusion_matrix(y, y_pred)
    precision = precision_score(y, y_pred, pos_label='1')
    recall = recall_score(y, y_pred, pos_label='1')
    f1 = f1_score(y, y_pred, pos_label='1')
    roc_auc = roc_auc_score(y, y_pred)
    best_parameters = grid_search.best_params_

    selected_features = X.columns[best_model.named_steps['feature_selector'].get_support()]

    resultat_compile[f'Random forest_{localite}_{spei}_{type_sec}']={
        #'Selected Features': selected_features,
        'Mean Accuracy': round(mean_accuracy,3),
        'Precision': round(precision,3),
        'Recall': round(recall,3),
        'F1 Score': round(f1,3),
        'ROC AUC': round(roc_auc,3),
        'loss':'-',
        'mse':'-',
        'mae':'-'
    }
    
    #Metrics dans un fichier
    columns = ['Algo', 'Spei', 'Type_Sec', 'Station', 'Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC AUC']
    
    # Création du DataFrame avec des colonnes vides
    df = pd.DataFrame(columns=columns)
    
    metrics_obtenu=['Random forest',spei,type_sec,localite,mean_accuracy, precision,recall,f1,roc_auc]
    # Ajout d'une nouvelle ligne au DataFrame
    df.loc[len(df)] =metrics_obtenu    
    result_file_csv = os.path.join(dir_resultat, f'rf_{spei}_{type_sec}_{localite}.csv')
    result_file_excel = os.path.join(dir_resultat, f'rf_{spei}_{type_sec}_{localite}.xlsx')
    
    df.to_csv(result_file_csv,index=False)
    df.to_excel(result_file_excel,index=False)

    columns_sel_feat=['Algo', 'Spei', 'Type_Sec', 'Station','Variable_1','Variable_2','Variable_3','Variable_4','Variable_5']
    df_sel = pd.DataFrame(columns=columns_sel_feat)
    variable1=selected_features[0]
    variable2=selected_features[1]
    variable3=selected_features[2]
    variable4=selected_features[3]
    variable5=selected_features[4]
    
    select_element=['Random forest', spei, type_sec, localite, variable1, variable2, variable3, variable4, variable5]
    # Ajout d'une nouvelle ligne au DataFrame
    df_sel.loc[len(df_sel)] =select_element   
    selected_features_file_csv = os.path.join(dir_resultat_sel_var, f'rf_{spei}_{type_sec}_{localite}.csv')
    df_sel.to_csv(selected_features_file_csv, index=False)
    selected_features_file_excel = os.path.join(dir_resultat_sel_var, f'rf
                                                _{spei}_{type_sec}_{localite}.xlsx')
    df_sel.to_excel(selected_features_file_excel, index=False)
    
    
    return best_model,  best_parameters


In [ ]:
def train_and_evaluate_boost(df_data,spei, type_sec,localite):
    
    
    df=df_data.copy()
    if 'Decade' in df.columns:
        df.drop('Decade', axis=1, inplace=True)
    df = df.drop(['Station', 'Year', 'Date'], axis=1)
    df['Label Secheresse'] = df['Label Secheresse'].astype(str)

    X = df.drop('Label Secheresse', axis=1)
    y = df['Label Secheresse']

    # Create a pipeline with data preprocessing, feature selection, and classifier
    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        #('scaler', StandardScaler()),
        ('feature_selector', SelectFromModel(GradientBoostingClassifier(random_state=42),threshold='median')),
        ('classifier', GradientBoostingClassifier(random_state=42))
    ])

    param_grid = {
        'classifier__n_estimators': [50, 100, 200],
        'classifier__learning_rate': [0.01, 0.1, 0.2],
        'classifier__max_depth': [3, 4, 5],
        'classifier__min_samples_split': [2, 5, 10],
        'classifier__min_samples_leaf': [1, 2, 4]
    }

    grid_search = GridSearchCV(pipeline, param_grid, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
                               n_jobs=-1, verbose=2)

    grid_search.fit(X, y)

    best_model = grid_search.best_estimator_

    k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(best_model, X, y, cv=k_fold, scoring='accuracy')
    mean_accuracy = scores.mean()

    # Calculate additional metrics
    y_pred = best_model.predict(X)
    confusion = confusion_matrix(y, y_pred)
    precision = precision_score(y, y_pred, pos_label='1')
    recall = recall_score(y, y_pred, pos_label='1')
    f1 = f1_score(y, y_pred, pos_label='1')
    roc_auc = roc_auc_score(y, y_pred)
    best_parameters = grid_search.best_params_

    selected_features = X.columns[best_model.named_steps['feature_selector'].get_support()]
    resultat_compile[f'Gradient boosting_{localite}_{spei}_{type_sec}']={
        #'Selected Features': selected_features,
        'Mean Accuracy': round(mean_accuracy,3),
        'Precision': round(precision,3),
        'Recall': round(recall,3),
        'F1 Score': round(f1,3),
        'ROC AUC': round(roc_auc,3),
        'loss':'-',
        'mse':'-',
        'mae':'-'
    }
    
    columns = ['Algo', 'Spei', 'Type_Sec', 'Station', 'Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC AUC']
    
    # Création du DataFrame avec des colonnes vides
    df = pd.DataFrame(columns=columns)
    
    metrics_obtenu=['Gradient Boost',spei,type_sec,localite,mean_accuracy, precision,recall,f1,roc_auc]
    # Ajout d'une nouvelle ligne au DataFrame
    df.loc[len(df)] =metrics_obtenu    
    result_file_csv = os.path.join(dir_resultat, f'gb_{spei}_{type_sec}_{localite}.csv')
    result_file_excel = os.path.join(dir_resultat, f'gb_{spei}_{type_sec}_{localite}.xlsx')
    
    df.to_csv(result_file_csv,index=False)
    df.to_excel(result_file_excel,index=False)
    
        #Variables selectionnées dans un fichier
    #selected_features_df = pd.DataFrame({'Selected Features': selected_features})
    columns_sel_feat=['Algo', 'Spei', 'Type_Sec', 'Station','Variable_1','Variable_2','Variable_3','Variable_4','Variable_5']
    df_sel = pd.DataFrame(columns=columns_sel_feat)
    variable1=selected_features[0]
    variable2=selected_features[1]
    variable3=selected_features[2]
    variable4=selected_features[3]
    variable5=selected_features[4]
    
    select_element=['Gradient Boost', spei, type_sec, localite, variable1, variable2, variable3, variable4, variable5]
    df_sel.loc[len(df_sel)] =select_element  
    selected_features_file_csv = os.path.join(dir_resultat_sel_var, f'gb_{spei}_{type_sec}_{localite}.csv')
    df_sel.to_csv(selected_features_file_csv, index=False)
    selected_features_file_excel = os.path.join(dir_resultat_sel_var, f'gb_{spei}_{type_sec}_{localite}.xlsx')
    df_sel.to_excel(selected_features_file_excel, index=False)
    
    
    return best_model, best_parameters

In [ ]:
def create_rnn_model(X_train, y_train, X_valid, y_valid, X_test, y_test, spei, type_sec, localite):
    input_shape = X_train[0].shape  # La forme d'entrée est déterminée par les données d'entraînement
    model = keras.Sequential([
        keras.layers.LSTM(64, input_shape=input_shape),
        keras.layers.Dense(1, activation='sigmoid')  # Couche de sortie avec une seule unité et activation sigmoid pour la classification binaire
    ])

    # Compilez le modèle avec la perte et l'optimiseur spécifiés
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_valid, y_valid))

    # Évaluez le modèle sur les données de test
    y_pred_proba = model.predict(X_test)
    y_pred = (y_pred_proba > 0.5).astype(int)  # Convertir les probabilités en classes binaires (0 ou 1)

    # Calculez les métriques
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)

    # Calculez le Mean Squared Error (MSE) et le Mean Absolute Error (MAE)
    #mse = tf.keras.losses.mean_squared_error(y_test, y_pred_proba).numpy()
    #mae = tf.keras.losses.mean_absolute_error(y_test, y_pred_proba).numpy()


    resultat_compile[f'RNN_{localite}_{spei}_{type_sec}'] = {
        'Mean Accuracy': round(np.mean([accuracy]), 3),
        'Precision': round(precision, 3),
        'Recall': round(recall, 3),
        'F1 Score': round(f1, 3),
        'ROC AUC': round(roc_auc, 3),
        #'loss': round(loss, 3),
        #'mse': round(np.mean([mse]), 3),
        #'mae': round(np.mean([mae]), 3)
    }
    columns = ['Algo', 'Spei', 'Type_Sec', 'Station', 'Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC AUC']

    # Création du DataFrame avec des colonnes vides
    df = pd.DataFrame(columns=columns)

    metrics_obtenu = ['RNN', spei, type_sec, localite, np.mean([accuracy]), round(precision, 3), round(recall, 3),
                      round(f1, 3), round(roc_auc, 3)]
    # Ajout d'une nouvelle ligne au DataFrame
    df.loc[len(df)] = metrics_obtenu
    result_file_csv = os.path.join(dir_resultat, f'rnn_{spei}_{type_sec}_{localite}.csv')
    result_file_excel = os.path.join(dir_resultat, f'rnn_{spei}_{type_sec}_{localite}.xlsx')

    df.to_csv(result_file_csv, index=False)
    df.to_excel(result_file_excel, index=False)

    return model

In [ ]:
def prepare_data_for_rnn(df_data, feature_columns, time_columns, target_column, sequence_length, test_size=0.2, valid_size=0.5):
    dataframe=df_data.copy()
    # Étape 1 : Triez par la colonne temporelle
    #dataframe.sort_values(by=time_columns [ 'Station',time_columns], ascending=[True, True], inplace=True)
    if 'Decade' in dataframe.columns:
        dataframe.sort_values(by=[ "Year", "Month", "Decade"], ascending=[True, True, True], inplace=True)
    else:
        dataframe.sort_values(by=["Year", "Month"], ascending=[True, True], inplace=True)



    # Étape 2 : Sélection des caractéristiques et de la cible
    features = feature_columns
    target = target_column
    dataframe[target]=dataframe[target].astype(int)

    # Étape 3 : Création de séquences temporelles
    X, y = [], []

    for i in range(len(dataframe) - sequence_length):
        X.append(dataframe[features].values[i:i+sequence_length])
        y.append(dataframe[target].values[i+sequence_length])

    X = np.array(X)
    y = np.array(y)

    # Aplatir les données
    X = X.reshape(X.shape[0], -1)  # Aplatissement en 2D

    # Étape 4 : Normalisation
    scaler = StandardScaler()  #MinMaxScaler()
    X = scaler.fit_transform(X)

    # Étape 5 : Division des données
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=test_size, shuffle=False)
    X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=valid_size, shuffle=False)

    # Étape 6 : Préparation des données pour le modèle RNN
    X_train = X_train.reshape(X_train.shape[0], sequence_length, len(features))
    X_valid = X_valid.reshape(X_valid.shape[0], sequence_length, len(features))
    X_test = X_test.reshape(X_test.shape[0], sequence_length, len(features))

    return X_train, y_train, X_valid, y_valid, X_test, y_test

### ENTRAINEMENT SUR LES DONNNEES DECADAIRES

#### Extreme

In [ ]:

data_ext_1dek=load_data_dek(str(Config.DATASET_DIR) + '/' + str(Config.DATA_1DEK_EXTR))

In [ ]:
data_ext_1dek.head()

In [ ]:
data_ext_1dek['Label Secheresse'].unique()

#### RandonForrest

In [ ]:
grouped_data = data_ext_1dek.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_model_ranf(group_data, '1decade', 'extr', station)

#### BOOST

In [ ]:
grouped_data = data_ext_1dek.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_boost(group_data, '1decade', 'extr', station)

In [ ]:
resultat_compile

#### RNN

In [ ]:
data_ext_1dek.head()

In [ ]:
grouped_data = data_ext_1dek.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    print(f"Processing station: {station}")
    X_train, y_train, X_valid, y_valid, X_test, y_test = prepare_data_for_rnn(group_data, feature_columns=feature_rnn_dek, time_columns=["Year", "Month", "Decade"], target_column="Label Secheresse", sequence_length=5)
    create_rnn_model(X_train, y_train, X_valid, y_valid, X_test, y_test, '1decade', 'extr', station)
    print(f"Processing complete for station: {station}")

#### Severe

In [ ]:
data_sev_1dek=load_data_dek(str(Config.DATASET_DIR) + '/' + str(Config.DATA_1DEK_SEV))

#### RandonForrest

In [ ]:
grouped_data = data_sev_1dek.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_model_ranf(group_data, '1decade', 'sev', station)


#### BOOST

In [ ]:
grouped_data = data_sev_1dek.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_boost(group_data, '1decade', 'sev', station)


#### RNN

In [ ]:
grouped_data = data_sev_1dek.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    X_train, y_train, X_valid, y_valid, X_test, y_test = prepare_data_for_rnn(data_sev_1dek, feature_columns=feature_rnn_dek,time_columns=["Year", "Month", "Decade"], target_column="Label Secheresse", sequence_length=5)
    create_rnn_model(X_train, y_train,X_valid, y_valid,X_test, y_test,'1decade','sev',station)

### ENTRAINEMENT SPEI 1-MOIS

#### Extreme

In [ ]:
data_ext_1mois=load_data_mon(str(Config.DATASET_DIR) + '/' + str(Config.DATA_1MON_EXTR))

#### RandonForrest

In [ ]:
grouped_data = data_ext_1mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_model_ranf(group_data, '1mois', 'extr', station)

#### BOOST

In [ ]:
grouped_data = data_ext_1mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_boost(group_data, '1mois', 'extr', station)


#### RNN

In [ ]:
grouped_data = data_ext_1mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    X_train, y_train, X_valid, y_valid, X_test, y_test = prepare_data_for_rnn(data_ext_1mois, feature_columns=feature_rnn_mon,time_columns=["Year", "Month"], target_column="Label Secheresse", sequence_length=5)
    create_rnn_model(X_train, y_train,X_valid, y_valid,X_test, y_test,'1mois','extr',station)

#### Severe

In [ ]:
data_sev_1mois=load_data_mon(str(Config.DATASET_DIR) + '/' + str(Config.DATA_1MON_SEV))

#### RandonForrest

In [ ]:
grouped_data = data_sev_1mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_model_ranf(group_data, '1mois', 'sev', station)

#### BOOST

In [ ]:
grouped_data = data_sev_1mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_boost(group_data, '1mois', 'sev', station)


#### RNN

In [ ]:
grouped_data = data_sev_1mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    X_train, y_train, X_valid, y_valid, X_test, y_test = prepare_data_for_rnn(data_sev_1mois, feature_columns=feature_rnn_mon,time_columns=["Year", "Month"], target_column="Label Secheresse", sequence_length=5)
    create_rnn_model(X_train, y_train,X_valid, y_valid,X_test, y_test,'1mois','sev',station)

### ENTRAINEMENT SPEI 3-MOIS

#### Extreme

In [ ]:

data_ext_3mois=load_data_mon(str(Config.DATASET_DIR) + '/' + str(Config.DATA_3MON_EXTR))

#### RandonForrest

In [ ]:
grouped_data = data_ext_3mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_model_ranf(group_data, '3mois', 'extr', station)


#### BOOST

In [ ]:
grouped_data = data_ext_3mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_boost(group_data, '3mois', 'extr', station)

#### RNN

In [ ]:
grouped_data = data_ext_3mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    X_train, y_train, X_valid, y_valid, X_test, y_test = prepare_data_for_rnn(data_ext_3mois, feature_columns=feature_rnn_mon,time_columns=["Year", "Month"], target_column="Label Secheresse", sequence_length=5)
    create_rnn_model(X_train, y_train,X_valid, y_valid,X_test, y_test,'3mois','extr',station)

#### Severe

In [ ]:
data_sev_3mois=load_data_mon(str(Config.DATASET_DIR) + '/' + str(Config.DATA_3MON_SEV))

#### RandonForrest

In [ ]:
grouped_data = data_sev_3mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_model_ranf(group_data, '3mois', 'sev', station)

#### BOOST

In [ ]:
grouped_data = data_sev_3mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_boost(group_data, '3mois', 'sev', station)

#### RNN

In [ ]:
grouped_data = data_sev_3mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    X_train, y_train, X_valid, y_valid, X_test, y_test = prepare_data_for_rnn(data_sev_3mois, feature_columns=feature_rnn_mon,time_columns=["Year", "Month"], target_column="Label Secheresse", sequence_length=5)
    create_rnn_model(X_train, y_train,X_valid, y_valid,X_test, y_test,'3mois','sev',station)

### ENTRAINEMENT SPEI 6-MOIS

#### Extreme

In [ ]:

data_ext_6mois=load_data_mon(str(Config.DATASET_DIR) + '/' + str(Config.DATA_6MON_EXTR))

In [ ]:
data_ext_6mois.head()

#### RandonForrest

In [ ]:
grouped_data = data_ext_6mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_model_ranf(group_data, '6mois', 'extr', station)


#### BOOST

In [ ]:
grouped_data = data_ext_6mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_boost(group_data, '6mois', 'extr', station)

#### RNN

In [ ]:
data_ext_6mois.head()

In [ ]:
grouped_data = data_ext_6mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    X_train, y_train, X_valid, y_valid, X_test, y_test = prepare_data_for_rnn(data_ext_6mois, feature_columns=feature_rnn_mon,time_columns=["Year", "Month"], target_column="Label Secheresse", sequence_length=5)
    create_rnn_model(X_train, y_train,X_valid, y_valid,X_test, y_test,'6mois','extr',station)

#### Severe

In [ ]:
data_sev_6mois=load_data_mon(str(Config.DATASET_DIR) + '/' + str(Config.DATA_6MON_SEV))

#### RandonForrest

In [ ]:
#rand_sev_6mois=train_and_evaluate_model_ranf(data_sev_6mois)
grouped_data = data_sev_6mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_model_ranf(group_data, '6mois', 'sev', station)

#### BOOST

In [ ]:
grouped_data = data_sev_6mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    train_and_evaluate_boost(group_data, '6mois', 'sev', station)

#### RNN

In [ ]:
grouped_data = data_sev_6mois.groupby('Station')

# Parcourez chaque groupe et appliquez la fonction
for station, group_data in grouped_data:
    #data_by_st = group_data.drop(['station'], axis=1)  # Assurez-vous de supprimer la colonne 'station' si elle ne fait pas partie de vos fonctionnalités
    X_train, y_train, X_valid, y_valid, X_test, y_test = prepare_data_for_rnn(data_sev_6mois, feature_columns=feature_rnn_mon,time_columns=["Year", "Month"], target_column="Label Secheresse", sequence_length=5)
    create_rnn_model(X_train, y_train,X_valid, y_valid,X_test, y_test,'6mois','sev',station)

### ENTRAINEMENT SPEI 1-MOIS

In [ ]:
resultat_compile

In [ ]:
result=pd.DataFrame(resultat_compile)

In [ ]:
result.to_csv('./resultat.csv')
result.to_excel('./resultat.xlsx')